In [2]:
#outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/single/"
in_files = {}
outdir = "/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes"
#cloneID = "0_C6"


In [1]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import mplh.fig_utils as fu
import numpy as np
import os
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom

import pandas as pd
import seaborn as sns
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

from svgutils.compose import *


import svgutils.transform as sg
import sys 

fig_utils from mplh


In [122]:

"""
based on https://neuroscience.telenczuk.pl/?p=331
"""

import svgutils.transform as sg


class Svg(object):
    "svg files with a data object (the svg), width, height and coordinates"

    def __init__(self, data, dim, coords, name, global_scale=0.5):
        self.data = data
        self.width = dim[0]*global_scale
        self.height = dim[1]*global_scale
        self.x = coords[0]
        self.y = coords[1]
        self.global_scale = global_scale
        self.name = name

    def scale_width_to_reference(self, reference_width):
        """Proportionally scale the image to a given width."""
        scalings_factor = reference_width / self.width
        self.data.moveto(0, 0, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor

    def scale_height_to_reference(self, reference_height):
        """Proportionally scale the image to a given height."""
        scalings_factor = reference_height / self.height
        self.data.moveto(0, 0)#, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor
        
    def scale_by_factor(self, scalings_factor):
        """Proportionally scale image by a scaling factor."""
        self.data.moveto(0, 0, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor

    def move(self, x, y):
        """Move the coordinates of an image."""
        self.data.moveto(x, y)
        self.x = x
        self.y = y


def get_size(svg_file):
    """Naively parse the svg text file to get the width and height."""
    with open(svg_file) as svg:
        for line in svg:
            if line.startswith('<svg'):
                paramdict = {e.split('="')[0]: e.split('="')[1] for e in line[5:-2].split('" ')}
                break
    return int(paramdict["width"].replace('pt', '')), int(paramdict["height"].replace('pt', ''))



def rescale_height(svgs, curr_keys, ref_key):
    """Change the dimensions of the images to the desired combinations."""
    for key in curr_keys:
        svgs[key].scale_height_to_reference(svgs[ref_key].height)
    assert svgs[ref_key].height == svgs[curr_keys[0]].height == svgs[curr_keys[-1]].height

#     left_height = sum([svgs[i].height for i in ['A', 'B', 'C', 'D']])
#     right_height = sum([svgs[i].height for i in ['E', 'F', ]])
#     scalings_factor = right_height / left_height
#     svgs['A'].scale_by_factor(scalings_factor)
#     svgs['B'].scale_by_factor(scalings_factor)
#     svgs['C'].scale_by_factor(scalings_factor)
#     svgs['D'].scale_by_factor(scalings_factor)
#     assert sum([svgs[i].height for i in ['A', 'B', 'C', 'D']]
#                ) == sum([svgs[i].height for i in ['E', 'F', ]])

    
def change_positions(df, svgs, widths, heights):
    """Move the images to the desired positions."""
    for i_row, row in enumerate(df.index):
        for i_col, col in enumerate(df.columns):
            curr = df.loc[row, col].split(".svg")[0]
            svgs[curr].move(sum(widths[:i_col]), sum(heights[:i_row]))
    

def letter_annotations(svgs, df=None, only_grid_titles=True, figure_names=None):
    if only_grid_titles:
        all_texts = []
        for ind in df.index:
            value = svgs[df.loc[ind].iloc[0].split(".svg")[0]]
            all_texts.append(sg.TextElement(value.x, value.y, ind, size=15, weight="bold"))
        for col in df.columns:
            nm = figure_names[col]
            value = svgs[df[col].iloc[0].split(".svg")[0]]
            all_texts.append(sg.TextElement(value.x, value.y-10, nm, size=15, weight="bold"))    
        return all_texts
    """Add letters based on the location of the images."""
    return [sg.TextElement(value.x, value.y, value.name, size=15, weight="bold")
            for key, value in svgs.items()]
#     return [sg.TextElement(value.x + 10, value.y + 15, value.name, size=15, weight="bold")
#             for key, value in svgs.items()]


def files_to_svg_dict(df, scale=0.5):
    """Convert a list of images to a dictionary.
    Mapping the image basename to the Svg class instance,
    setting the dimensions based on sizes and coordinates (0,0) by default
    """
    out = {}
    for row in df.index:
        for col in df.columns:
            s = df.loc[row, col]
            key = df.loc[row, col].split(".svg")[0]
            name = f"{row} {col.replace('dendro', '').replace('.svg', '')}"
            out[key] = Svg(
                data=sg.fromfile(s).getroot().scale(scale),
                dim=get_size(s),
                coords=(0, 0),
                global_scale=scale,
                name=name)
    return out
#     return {
#         s.split('.svg')[0]: Svg(
#             data=sg.fromfile(s).getroot(),
#             dim=get_size(s),
#             coords=(0, 0))
#         for s in files}




# def main(out_f):
#     #svgs = files_to_svg_dict(["A.svg", "B.svg", "C.svg", "D.svg", "E.svg", "F.svg"])
#     rescale_height(svgs)
#     change_positions(svgs)
#     full_width = sum([svgs[i].width for i in ['A', 'E']])
#     full_height = sum([svgs[i].height for i in ['A', 'B', 'C', 'D']])
#     fig = sg.SVGFigure(full_width, full_height)
#     text = letter_annotations(svgs)
#     fig.append([s.data for s in svgs.values()])
#     fig.append(text)
#     fig.save(out_f)



def main_df(df, out_f, scale=0.25, figure_names=None):
    svgs = files_to_svg_dict(df, scale=scale)#df.values().flatten())
    
    for don, val in df.iterrows():
        curr_init_files = []
        curr_ref = val.iloc[0].split('.svg')[0] #svgs[val.iloc[0].split('.svg')[0]]
        curr_keys = [x.split('.svg')[0] for x in val.values] 
        rescale_height(svgs, curr_keys, curr_ref)
    
    col_max_widths = []

    for col in df.columns:
        curr_col_max = 0
        for ind in df.index:
            tmp_width = svgs[df.loc[ind, col].split(".svg")[0]].width
            if tmp_width > curr_col_max:
                curr_col_max = tmp_width
        col_max_widths.append(curr_col_max)

    row_max_heights = [] #should be the same in theory  
    for ind in df.index:
        curr_row_max = 0
        for col in df.columns:
            tmp_height = svgs[df.loc[ind, col].split(".svg")[0]].height
            if tmp_height > curr_row_max:
                curr_row_max = tmp_height
        row_max_heights.append(curr_row_max)
    
    change_positions(df, svgs, col_max_widths, row_max_heights)
    
    full_width = sum(col_max_widths) #[svgs[i].width for i in ['A', 'E']])
    full_height = sum(row_max_heights) #[svgs[i].height for i in ['A', 'B', 'C', 'D']])
    fig = sg.SVGFigure(full_width, full_height)
    text = letter_annotations(svgs, df=df, figure_names=figure_names)
    fig.append([s.data for s in svgs.values()])
    fig.append(text)
    fig.save(out_f)
 
    cmd = f'inkscape --file={out_f} -d 300 --export-area-drawing --without-gui --export-png={out_f.replace(".svg", ".png")}' # --export-area-drawing --without-gui --export-pdf=output.pdf'
    print(cmd)
    !{cmd}
    
    cmd = f'inkscape --file={out_f} --export-area-drawing --without-gui --export-pdf={out_f.replace("svg", "pdf")} '
    print(cmd)
    !{cmd}
    
    




In [123]:

main_df(dendro_don_subfig_df, join(outdir, "dendrogram_barcodes.svg"), scale=0.25, figure_names=dendro_figure_names)

inkscape --file=/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.svg -d 300 --export-area-drawing --without-gui --export-png=/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.png
DPI: 300
Background RRGGBBAA: ffffff00
Area 0:-2240:625.562:121.396 exported to 1955 x 7379 pixels (300 dpi)
Bitmap saved as: /data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.png
inkscape --file=/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.svg --export-area-drawing --without-gui --export-pdf=/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.pdf 


In [36]:
def merge_svg_panels(df, out_f):
    fig = sg.SVGFigure("16cm", "6.5cm")

    all_panels = []
    don_txts = []
    #roots = []

    x_shift = 280
    y_shift = 280
    row_v = 0
    for ind, val in df.iterrows():
        curr_init_files = []
        #curr_plots = []
        don_txts.append(ind)
        for ind2, elem in enumerate(val):
            curr_text = f"donor{ind}_fig{ind2}"
            print('curr_text', curr_text)
            curr_init_files.append(Panel(
                SVG(elem), Text(curr_text, size=12, weight="bold")))
            #curr_plots.append(curr_plot)
        all_panels += curr_init_files
        row_v += 1

    Figure("32cm", "13cm", *all_panels).tile(df.shape[0], 
                                             df.shape[1]).save(out_f)
    cmd = f'inkscape --export-pdf={out_f.replace("svg", "pdf")} {out_f}'
    print(cmd)
    !{cmd}
    return all_panels, all_txts





In [34]:
def merge_svg(df, out_f):
    fig = sg.SVGFigure("16cm", "6.5cm")

    all_fs = []
    don_txts = []
    #roots = []

    x_shift = 280
    y_shift = 280
    row_v = 0
    for ind, val in df.iterrows():
        curr_init_files = []
        curr_plots = []
        don_txts.append(ind)
        for ind2, elem in enumerate(val):
            curr_init_files.append(sg.fromfile(elem))
            curr_plot = curr_init_files[-1].getroot()
            if len(curr_init_files) != 1:
                print('len curr_init_files', len(curr_init_files))
                print("row_v", row_v)
                curr_plot.moveto(280*(len(curr_init_files)), 280*row_v) #, scale=0.75)
            curr_plots.append(curr_plot)
        all_fs += curr_plots
        row_v += 1

    all_txts = []
    for d, val in enumerate(don_txts):
        all_txts.append(sg.TextElement(0,280*d, val, size=12, weight="bold"))

    for ind, col in enumerate(df.columns):
        all_txts.append(sg.TextElement(280*ind, 0, col, size=12, weight="bold"))
        
    fig.append(all_fs)
    fig.append(all_txts)
    fig.save(out_f) #join(outdir, "clone_var_supplementary.svg"))
    
    cmd = f'inkscape --export-pdf={out_f.replace("svg", "pdf")} {out_f}'
    print(cmd)
    !{cmd}
    return all_fs, all_txts



## 1. clone_var supplementary

In [3]:
clone_var_donor_dirs = {
    "chip_b1": "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30",
    "chip_b2": "/data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/",
    "chip_a1": "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30",
    "cd34": "/data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30"
}

n_donors = {
    "chip_b1": 2,
    "chip_b2": 2,
    "chip_a1": 5,
    "cd34":4
}

In [4]:
for run in clone_var_donor_dirs:
    for d in np.arange(n_donors[run]):
        curr_pdfs = glob(join(clone_var_donor_dirs[run], f"donor{d}",  "*.pdf"))
        for in_f in curr_pdfs:
        #in_f = join(clone_var_donor_dirs, "best_params.pdf")
            out_f = in_f.replace(".pdf", ".svg")
            cmd = f"inkscape -l {out_f} {in_f}" 
            print(cmd)
            !{cmd}

inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/top10perc_param_group_clusters.svg /data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/top10perc_param_group_clusters.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/elbow_multi_obj_nclust.svg /data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001

inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/multi_clusters_nclust_5.svg /data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/multi_clusters_nclust_5.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/best_params.svg /data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_

inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/multi_clusters_nclust_3.svg /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/multi_clusters_nclust_3.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor1/top10perc_param_group_clusters.svg /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblackli

inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/elbow_multi_obj_nclust.svg /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/elbow_multi_obj_nclust.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/top_param_group_results.svg /data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/m

inkscape -l /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/best_params.svg /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/best_params.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor0/multi_clusters_nclust_4.svg /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched

inkscape -l /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/top10perc_param_group_clusters.svg /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/top10perc_param_group_clusters.pdf
inkscape -l /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/donor3/elbow_multi_obj_nclust.svg /data/Mito_Trace/output/pipeline/v04/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_

## Loop through and align svgs, with each donor in horizontal

In [5]:
indexes = []
for run in clone_var_donor_dirs:
    for d in np.arange(n_donors[run]):
        indexes.append(join(f"{run}_donor{d}"))
        
don_row_subfig_col_df = pd.DataFrame(index=indexes,columns = ["best_params.svg",
                                                      "top10perc_param_group_clusters.svg",
                                                      "elbow_multi_obj_nclust.svg",
                                                      "top_param_groups_clone_vars.svg"])
for run in clone_var_donor_dirs:
    for d in np.arange(n_donors[run]):
        curr_ind = f"{run}_donor{d}"
        
        for f_name in don_row_subfig_col_df.columns:
            curr_f = (join(clone_var_donor_dirs[run], f"donor{d}",  f_name))
            don_row_subfig_col_df.loc[curr_ind, f_name] = curr_f
            
don_row_subfig_col_df

,best_params.svg,top10perc_param_group_clusters.svg,elbow_multi_obj_nclust.svg,top_param_groups_clone_vars.svg
chip_b1_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
chip_b1_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
chip_b2_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...
chip_b2_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...
chip_a1_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor2,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor3,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor4,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
cd34_donor0,/data/Mito_Trace/output/pipeline/v04/cd34norm/...,/data/Mito_Trace/output/pipeline/v04/cd34norm/...,/data/Mito_Trace/output/pipeline/v04/cd34norm/...,/data/Mito_Trace/output/pipeline/v04/cd34norm/...


In [18]:
fig = sg.SVGFigure("16cm", "6.5cm")

all_fs = []
don_txts = []
#roots = []

x_shift = 280
y_shift = 280
row_v = 0
for ind, val in don_row_subfig_col_df.iterrows():
    curr_fs = []
    curr_plots = []
    don_txts.append(ind)
    for ind2, elem in enumerate(val):
        curr_fs.append(sg.fromfile(elem))
        curr_plot = curr_fs[-1].getroot()
        if len(curr_fs) != 1:
            print('len curr_fs', len(curr_fs))
            print("row_v", row_v)
            curr_plot.moveto(280*(len(curr_fs)), 280*row_v) #, scale=0.75)
        curr_plots.append(curr_plot)
    all_fs += curr_plots
    row_v += 1

all_txts = []
for d, val in enumerate(don_txts):
    all_txts.append(sg.TextElement(0,280*d, val, size=12, weight="bold"))

for ind, col in enumerate(don_row_subfig_col_df.columns):
    all_txts.append(sg.TextElement(280*ind, 0, col, size=12, weight="bold"))


all_txts

fig.append(all_fs)
fig.append(all_txts)
fig.save(join(outdir, "clone_var_supplementary.svg"))

cmd = f'inkscape --export-pdf={join(outdir, "clone_var_supplementary.pdf")} {join(outdir, "clone_var_supplementary.svg")}'
print(cmd)
!{cmd}

outdir

len curr_fs 2
row_v 0
len curr_fs 3
row_v 0
len curr_fs 4
row_v 0
len curr_fs 2
row_v 1
len curr_fs 3
row_v 1
len curr_fs 4
row_v 1
len curr_fs 2
row_v 2
len curr_fs 3
row_v 2
len curr_fs 4
row_v 2
len curr_fs 2
row_v 3
len curr_fs 3
row_v 3
len curr_fs 4
row_v 3
len curr_fs 2
row_v 4
len curr_fs 3
row_v 4
len curr_fs 4
row_v 4
len curr_fs 2
row_v 5
len curr_fs 3
row_v 5
len curr_fs 4
row_v 5
len curr_fs 2
row_v 6
len curr_fs 3
row_v 6
len curr_fs 4
row_v 6
len curr_fs 2
row_v 7
len curr_fs 3
row_v 7
len curr_fs 4
row_v 7
len curr_fs 2
row_v 8
len curr_fs 3
row_v 8
len curr_fs 4
row_v 8
len curr_fs 2
row_v 9
len curr_fs 3
row_v 9
len curr_fs 4
row_v 9
len curr_fs 2
row_v 10
len curr_fs 3
row_v 10
len curr_fs 4
row_v 10
len curr_fs 2
row_v 11
len curr_fs 3
row_v 11
len curr_fs 4
row_v 11
len curr_fs 2
row_v 12
len curr_fs 3
row_v 12
len curr_fs 4
row_v 12


In [ ]:


# #create new SVG figure
# fig = sg.SVGFigure("16cm", "6.5cm")

# # load matpotlib-generated figures
# fig1 = sg.fromfile('sigmoid_fit.svg')
# fig2 = sg.fromfile('anscombe.svg')

# # get the plot objects
# plot1 = fig1.getroot()
# plot2 = fig2.getroot()
# plot2.moveto(280, 0, scale=0.5)

# # add text labels
# txt1 = sg.TextElement(25,20, "A", size=12, weight="bold")
# txt2 = sg.TextElement(305,20, "B", size=12, weight="bold")

# # append plots and labels to figure
# fig.append([plot1, plot2])
# fig.append([txt1, txt2])

# # save generated SVG files
# fig.save("fig_final.svg")



In [ ]:
don_row_subfig_col_df

## 2. Dendrograms: 

In [32]:
dendro_don_subfig_df = pd.DataFrame(index=indexes,columns = ["dendro.NoCondition.max2.AF.svg",
                                                             "dendro.NoCondition.max5.AF.svg",
                                                             "dendro.NoCondition.DP.svg"]
                                   )


for run in clone_var_donor_dirs:
    for d in np.arange(n_donors[run]):
        curr_ind = f"{run}_donor{d}"
        curr_dendro_f = join(clone_var_donor_dirs[run], "barcodes/btwnClones_dendro", f"donor{d}.")
        for f_name in dendro_don_subfig_df.columns:
            curr_f = curr_dendro_f + f_name
            dendro_don_subfig_df.loc[curr_ind, f_name] = curr_f
            assert(exists(curr_f))
dendro_don_subfig_df   


,dendro.NoCondition.max2.AF.svg,dendro.NoCondition.max5.AF.svg,dendro.NoCondition.DP.svg
chip_b1_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
chip_b1_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
chip_b2_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...
chip_b2_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...,/data/Mito_Trace/output/pipeline/v04/CHIP_b2/I...
chip_a1_donor0,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor1,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor2,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor3,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
chip_a1_donor4,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...,/data/Mito_Trace/output/pipeline/v04/CHIP_Inpu...
cd34_donor0,/data/Mito_Trace/output/pipeline/v04/cd34norm/...,/data/Mito_Trace/output/pipeline/v04/cd34norm/...,/data/Mito_Trace/output/pipeline/v04/cd34norm/...


In [39]:
dendro_don_subfig_df.iloc[5,0]

'/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro/donor1.dendro.NoCondition.max2.AF.svg'

In [37]:
merge_svg(dendro_don_subfig_df, join(outdir, "dendrogram_barcodes.svg"))

len curr_init_files 2
row_v 0
len curr_init_files 3
row_v 0
len curr_init_files 2
row_v 1
len curr_init_files 3
row_v 1
len curr_init_files 2
row_v 2
len curr_init_files 3
row_v 2
len curr_init_files 2
row_v 3
len curr_init_files 3
row_v 3
len curr_init_files 2
row_v 4
len curr_init_files 3
row_v 4
len curr_init_files 2
row_v 5
len curr_init_files 3
row_v 5
len curr_init_files 2
row_v 6
len curr_init_files 3
row_v 6
len curr_init_files 2
row_v 7
len curr_init_files 3
row_v 7
len curr_init_files 2
row_v 8
len curr_init_files 3
row_v 8
len curr_init_files 2
row_v 9
len curr_init_files 3
row_v 9
len curr_init_files 2
row_v 10
len curr_init_files 3
row_v 10
len curr_init_files 2
row_v 11
len curr_init_files 3
row_v 11
len curr_init_files 2
row_v 12
len curr_init_files 3
row_v 12
inkscape --export-pdf=/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.pdf /data/Mito_Trace/output/aggregate/CHIP_aggr/v04/results/enriched_barcodes/dendrogram_barcodes.

([<svgutils.transform.GroupElement at 0x7f357faf8fd0>,
  <svgutils.transform.TextElement at 0x7f357f166990>])

In [ ]:
# out_d["mean"] = f"{out_f}.mean.csv"
# out_d["dendro_csv"] = f"{out_f}.clones_dendro.csv"
# out_d["m02"] = f"{out_f}.dendro.NoCondition.max2.AF.png"
# out_d["m05"] = f"{out_f}.dendro.NoCondition.max5.AF.png"
# out_d["dp"] = f"{out_f}.dendro.NoCondition.DP.png"

## Concatenate vertically each panel

In [3]:
in_imgs = glob(outdir+"/*svg")
n_imgs = len(in_imgs)

in_imgs

['/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top_variants_distinct_pct_vs_other.svg',
 '/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top10_sig_hypergeo.svg',
 '/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top_variants_af.svg',
 '/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filte

## Convert pdf to svg when needed

## Each donor is a row

In [4]:
# cmd = f"gs -dNOPAUSE -sDEVICE=pdfwrite -sOUTPUTFILE=combine.pdf -dBATCH {join(outdir,'top_umap_clust_col.pdf')} {join(outdir,'top_umap_cond_col.pdf')} {join(outdir,'top_cluster_condition_ncells.pdf')}"
# cmd


In [5]:
import svgutils.transform as sg
import sys 


In [1]:
out_svg_f = join(outdir,'clone_shift_combine.svg')
out_pdf_f = join(outdir,'clone_shift_combine.pdf')

out_vars_svg_f = join(outdir,'clone_variants_combine.svg')
out_vars_pdf_f = join(outdir,'clone_variants_combine.pdf')

NameError: name 'join' is not defined

In [ ]:
f1_umap_clust = join(outdir,'top_umap_clust_col.svg')
f2_umap_cond = join(outdir,'top_umap_cond_col.svg')
f3_clone_clust = join(outdir,'top_cluster_condition_ncells.svg')
f4_hyper = join(outdir,'top10_sig_hypergeo.svg')

fvars_pct = join(outdir, "top_variants_distinct_pct_vs_other.svg")
fvars_pct_minus = join(outdir, "top_variants_pct_minus_other.svg")
fvars_vars_af = join(outdir, "top_variants_af.svg")
fvars_vars_af_box = join(outdir, "top_variants_in_clone_af_boxen.svg") 


In [ ]:
cmd = f"svg_stack.py --direction=h --margin=100 {f1_umap_clust} {f2_umap_cond} {f3_clone_clust} {f4_hyper} > {out_svg_f}"
print(cmd)
!{cmd}

cmd=f"inkscape --export-pdf={out_pdf_f} {out_svg_f}"
print(cmd)
!{cmd}

In [ ]:
cmd = f"svg_stack.py --direction=h --margin=100 {fvars_pct} {fvars_pct_minus} {fvars_vars_af_box} {fvars_vars_af} > {out_vars_svg_f}"
print(cmd)
!{cmd}

cmd=f"inkscape --export-pdf={out_vars_pdf_f} {out_vars_svg_f}"
print(cmd)
!{cmd}